In [28]:
import os
import numpy as np
from helpers import*
from models import *   
import isdead
import load_data
import nan_imputation

In [29]:
# Reload isdead.py
import importlib
importlib.reload(load_data)
importlib.reload(nan_imputation)

<module 'nan_imputation' from '/Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/nan_imputation.py'>

## Step 0 : Load the Data

In [30]:
repo_root = find_repo_root()
# load data
# Define the path to the Lifespan folder
data_path = os.path.join(repo_root, 'Data/Lifespan')

In [31]:
fdict = load_data.load_lifespan(data_path)

/Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/control/speeds_and_coordinates_20241016_16_updated.csv
File loaded as CSV.
/Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/control/speeds_and_coordinates_20241016_31_updated.csv
File loaded as CSV.
/Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/control/speeds_and_coordinates_20241016_34_updated.csv
File loaded as CSV.
/Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/control/speeds_and_coordinates_20241016_13_updated.csv
File loaded as CSV.
/Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/control/speeds_and_coordinates_20241016_32_updated.csv
File loaded as CSV.
/Users/louistschanz/Documents/EPFL-Cours/MA1/ML/Project-2/ML-Project-2/Data/Lifespan/control/speeds_and_coordinates_20241016_15_updated.csv
File loaded as CSV.
/Users/louistschanz/Documents/EPFL-Cours

## Step 1 : NaN imputation
> impute only on X and Y columns since only where there are NaN

In [32]:
nan_dict_xy = {}  # Dictionary to store the proportion of NaN values for each worm's X and Y data
for name, arr in fdict.items(): # Iterate over each worm in the dataset
    new_arr = arr[1:3, :]  # Shape: (2, number_of_frames)/ # Extract only the rows corresponding to X and Y data
    total_nans = np.sum(pd.isna(new_arr))  # Counts all NaN values in 'new_arr'
    total_elements = new_arr.shape[0] * new_arr.shape[1]  # 2 rows * number_of_frames = total number of elements
    nansxy = total_nans / total_elements # Compute the proportion of NaN values    
    nan_dict_xy.update({name: nansxy}) # Update the dictionary with the worm's name and the calculated proportion
# After the loop, 'nan_dict_xy' contains the proportion of NaN values for each worm

/var/folders/0_/l7m0lf291p947gwc6cflm9cw0000gn/T/ipykernel_6222/2993027071.py:6: RuntimeWarning: invalid value encountered in scalar divide
  nansxy = total_nans / total_elements # Compute the proportion of NaN values


In [33]:
nan_dict_xy

{'worm_0_control': 0.007798341116240922,
 'worm_1_control': 0.005546809889804612,
 'worm_2_control': 0.011186220946383925,
 'worm_3_control': 0.010126599023719127,
 'worm_4_control': 0.019829878427896762,
 'worm_5_control': 0.0002700867364262123,
 'worm_6_control': 0.035558848041485326,
 'worm_7_control': 0.022336724299869035,
 'worm_8_control': 0.024942785707671344,
 'worm_9_control': 0.016464487452541903,
 'worm_10_control': 0.002870636170015742,
 'worm_11_control': 0.017600836056248594,
 'worm_1_companyDrug': 0.001653592263833953,
 'worm_2_companyDrug': nan,
 'worm_3_companyDrug': 0.004881404362837829,
 'worm_4_companyDrug': 0.002006358613451863,
 'worm_5_companyDrug': 0.008506078605161853,
 'worm_6_companyDrug': 0.0013096450729564906,
 'worm_7_companyDrug': 0.0,
 'worm_8_companyDrug': 0.007255962853703385,
 'worm_9_companyDrug': 0.018182900533118145,
 'worm_10_companyDrug': 0.01293109150318151,
 'worm_12_companyDrug': 0.001703861468654505,
 'worm_13_companyDrug': 0.0059793896260235

# Step 1.1 : Identify where both X and Y are NaNs

In [40]:
nan_dict_cols = {}
for name,arr in fdict.items():
    new_arr = arr[2:4,:] #extract just X and Y to see nans
    nan_cols = np.isnan(new_arr).all(axis=0)
    number_of_nan_columns = np.sum(nan_cols)
    ratio = number_of_nan_columns / arr.shape[1]
    nan_dict_cols.update({name: ratio})

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [44]:
nan_dict_cols

{'worm_0_control': 0.015596682232481843,
 'worm_1_control': 0.011093619779609223,
 'worm_2_control': 0.02237244189276785,
 'worm_3_control': 0.020253198047438255,
 'worm_4_control': 0.039659756855793524,
 'worm_5_control': 0.0005401734728524246,
 'worm_6_control': 0.07111769608297065,
 'worm_7_control': 0.04467344859973807,
 'worm_8_control': 0.04988557141534269,
 'worm_9_control': 0.032928974905083806,
 'worm_10_control': 0.005741272340031484,
 'worm_11_control': 0.03520167211249719,
 'worm_1_companyDrug': 0.003307184527667906}

In [45]:
# Iterate over the dictionary
for name, lifespan_array in fdict.items():
    print(f"Processing {name}")
    lifespan_arrayxy = lifespan_array[2:4,:] # Extract rows for X and Y
    missing_sequences = nan_imputation.count_successive_missing(lifespan_arrayxy)
    for start, end, length in missing_sequences:
        print(f"  Missing sequence starts at column {start}, ends at column {end - 1}, length: {length}")

Processing worm_0_control
  Missing sequence starts at column 361, ends at column 368, length: 8
  Missing sequence starts at column 609, ends at column 610, length: 2
  Missing sequence starts at column 859, ends at column 899, length: 41
  Missing sequence starts at column 1018, ends at column 1076, length: 59
  Missing sequence starts at column 1127, ends at column 1154, length: 28
  Missing sequence starts at column 1178, ends at column 1208, length: 31
  Missing sequence starts at column 4077, ends at column 4093, length: 17
  Missing sequence starts at column 4127, ends at column 4189, length: 63
  Missing sequence starts at column 4744, ends at column 4768, length: 25
  Missing sequence starts at column 5469, ends at column 5562, length: 94
  Missing sequence starts at column 5693, ends at column 5719, length: 27
  Missing sequence starts at column 5727, ends at column 5920, length: 194
  Missing sequence starts at column 5922, ends at column 5924, length: 3
  Missing sequence s

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [20]:
# Rows to check for missing values (2:4 in zero-based indexing)
rows_to_check = slice(2, 4)  # Rows 2 and 3 not row 4

# Apply cut_array to each worm in the dataset
cut_lifespan_dict = {name: nan_imputation.cut_array(array, rows_to_check) for name, array in fdict.items()}

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [46]:
# Print the shape of the filtered arrays
for name, item in cut_lifespan_dict.items():
    print(f'{name} : {item.shape}')

NameError: name 'cut_lifespan_dict' is not defined